### Prepare dataset

As DOTA dataset is originally prepared for object detection/classification tasks, it is required to preprocess the data to be used for super-resolution task. This notebook converts the data into useable format.

In [ ]:
import pandas as pd
import glob
from shutil import copyfile, copytree
from PIL import Image
import os
from random import randrange

In [ ]:
"""
Read label and extract GSD
"""
files = glob.glob("dataset/train/labelTxt/*.txt")
GSD = []
file_name = []
for file in files:
    with open(file, 'r') as f:
        file_name.append(file.split('\\')[-1])
        f.readline() # ignore first line
        txt = f.readline()
        if "null" not in txt:
            GSD.append(float(txt[4:-1]))
        else:
            GSD.append(0.0)

In [ ]:
"""
Create pandas DataFrame for visualization
"""
df = pd.DataFrame({'filename': file_name, 'GSD': GSD})
df.describe()

In [ ]:
df.plot.hist(bins=50)

In [ ]:
"""
Identify the images whose GSD is between 0.1 and 0.15 to have images with similar height
"""
df1 = df[df['GSD'] < 0.15]
df2 = df1[df1['GSD'] > 0.1]
df2.plot.hist(bins=10)

In [ ]:
df2.describe()

In [ ]:
"""
Extract image and labels that are meeting the criteria above.
"""
for index, row in df2.iterrows():
    copyfile('dataset/train/images/'+row['filename'].replace('txt', 'png'), 'dataset/train/images_stage1/'+row['filename'].replace('txt', 'png'))
    copyfile('dataset/train/labelTxt/'+row['filename'], 'dataset/train/labelTxt_stage1/'+row['filename'])

In [ ]:
df2

In [ ]:
"""
Crop the image so that all images have same resolution
"""
def crop(path, file, height, width):
    im = Image.open(file)
    filename = file.split('\\')[-1]
    filename = filename[:-4]
    imgwidth, imgheight = im.size
    k = 0
    for i in range(30,imgheight-30,height):
        for j in range(30,imgwidth-30,width):
            if j+width > imgwidth or i+height > imgheight:
                continue
            box = (j, i, j+width, i+height)
            a = im.crop(box)
            a.save(os.path.join(path, f"{filename}-{k}.png"))
            k += 1

In [ ]:
"""
Perform cropping
"""
images_st1 = glob.glob("dataset/train/images_stage1/*.png")
for img in images_st1:
    crop('dataset/train/images_stage2', img, 1024, 1024)

In [ ]:
"""
Go through image and remove images where black area is included
We consider image are invalid if any 4 corner of image is black
"""
# copy to stage 3
copytree('dataset/train/images_stage2','dataset/train/images_stage3')

In [ ]:
images_st3 = glob.glob("dataset/train/images_stage3/*.png")
for img in images_st3:
    image = Image.open(img)
    width, height = image.size
    loc = ((0,0), (width-1,0), (0,height-1), (width-1,height-1))
    for x,y in loc:
        if sum(image.getpixel((x, y))) < 5:
            os.remove(img)
            break

In [ ]:
"""
Present some of high resolution images
"""
image_path = 'dataset/train/images_stage3/P0038-3.png'
image = Image.open(image_path)
image.show()

In [ ]:
"""
Create low resolution image out of high resolution images
"""
reduced_512 = image.resize((512,512), resample=Image.BICUBIC)
reduced_512.show()

In [ ]:
upsample_512 = reduced_512.resize((1024,1024), resample=Image.BICUBIC)
upsample_512.show()

In [ ]:
reduced_216 = image.resize((216,216), resample=Image.BICUBIC)
reduced_216.show()

In [ ]:
upsample_256 = reduced_216.resize((1024,1024), resample=Image.BICUBIC)
upsample_256.show()

#
# We will downsample to 216, and upsample to create low resolution image
#
NEAREST_256 = reduced_216.resize((1024,1024), resample=Image.NEAREST)
upsample_256.show()

In [ ]:
BOX_256 = reduced_216.resize((1024,1024), resample=Image.BOX)
upsample_256.show()

In [ ]:
BILINEAR_256 = reduced_216.resize((1024,1024), resample=Image.BILINEAR)
upsample_256.show()

In [ ]:
HAMMING_256 = reduced_216.resize((1024,1024), resample=Image.HAMMING)
upsample_256.show()

In [ ]:
"""
Create low resolution input 256x256
Also, create 1024 low res image with bicubic
"""
images_st3 = glob.glob("dataset/train/images_stage3/*.png")
resample_method = [Image.BICUBIC, Image.NEAREST, Image.BOX, Image.BILINEAR, Image.HAMMING]
for img in images_st3:
    image = Image.open(img)
    reduced_img = image.resize((256,256), resample=resample_method[randrange(5)])
    reduced_img.save(img.replace('images_stage3', 'images_stage5'))
    upsample_img = reduced_img.resize((1024,1024), resample=Image.BICUBIC)
    upsample_img.save(img.replace('images_stage3', 'images_stage4'))

In [ ]:
"""
Create 512 reference image for LAPSRN
"""
images_st3 = glob.glob("dataset/train/images_stage3/*.png")
resample_method = [Image.BICUBIC, Image.NEAREST, Image.BOX, Image.BILINEAR, Image.HAMMING]
for img in images_st3:
    image = Image.open(img)
    mid_img = image.resize((512,512), resample=Image.BICUBIC)
    mid_img.save(img.replace('images_stage3', 'images_stage6'))

Note that 

- images_stage3 => High Resolution Image
- images_stage4 => Low Resolution 1024x1024
- images_stage5 => Low Resolution 256x256
- images_stage6 => High Resolution 512 Image